#Diffie-Hellman Key Exchange between notebooks

This notebook allows other notebooks to exchange messages with it. Choose one notebook to be Alice's and the other to be Bob's. Run the cells up to the Scenario block, then work together to exchange a message.

In [ ]:
def xor(string, key):
  if isinstance(string,str) : string = bytearray(string, "utf-8")

  key_bytes = bytearray(str(key), "utf-8")
  encrypted_bytes = bytearray(len(string))

  # Loop through each byte of the string
  for i in range(len(string)):
    # Use modulo to ensure key wraps around if it's longer
    key_index = i % len(key_bytes)
    # XOR each byte with the corresponding byte from the key
    encrypted_bytes[i] = string[i] ^ key_bytes[key_index]

  return encrypted_bytes

In [ ]:
msg = b'''
# The following line is a secret message
There are oranges under the counter.
'''

In [ ]:
from cryptography.hazmat.primitives.asymmetric import dh

## Wormhole key exchange

In [ ]:
# Install dependencies
!pip3 install magic-wormhole twisted-ipython >/dev/null

# Define magic wormhole static variables
import wormhole
from wormhole.cli import public_relay
appid = "cis.highline.edu"
relay_url = public_relay.RENDEZVOUS_RELAY

# Wormhole uses Twisted Internet for asynchronous threads
from twisted.internet import reactor
%load_ext twisted_ipython
%autoawait twisted

In [ ]:
async def get_msg(code=None):
  w = wormhole.create(appid, relay_url, reactor)

  if code is None:
    w.allocate_code()
    code = await w.get_code()
  else:
    w.set_code(code)

  print(f"Getting message from wormhole {code}")

  msg = await w.get_message()
  result = await w.close()
  print(f"Connection closed: {result}")

  return msg

In [ ]:
async def send_msg(msg,code=None):
  w = wormhole.create(appid, relay_url, reactor)

  if code is None:
    w.allocate_code()
    code = await w.get_code()
  else:
    w.set_code(code)

  print(f"Sending message to wormhole {code}")

  versions = await w.get_versions()
  w.send_message(msg)
  result = await w.close()
  print(f"Connection closed: {result}")

  return code

##Scenario

Alice can initiate sending a message. She will need to tell Bob which wormhole address to monitor.

In [ ]:
#alice
import base64
from cryptography.hazmat.primitives import serialization

parameters = dh.generate_parameters(generator=2, key_size=2**9)
alice = parameters.generate_private_key()

pem = alice.public_key().public_bytes(
  encoding=serialization.Encoding.PEM,
  format=serialization.PublicFormat.SubjectPublicKeyInfo
)

code = await send_msg(pem,code=None)

bob_trusted = serialization.load_pem_public_key(await get_msg(code))

s = alice.exchange(bob_trusted)
print(f"Compute the shared secret as s=(B^a) mod p={s.hex()}")

await send_msg(base64.b64encode(xor(msg,s)),code)


Bob listens on the wormhole address and generates a private key based on the parameters passed through the tunnel.

In [ ]:
#bob
import base64
from cryptography.hazmat.primitives import serialization

print("What is the wormhole code?")
code = input()

# Create a wormhole and wait for a message
alice_trusted = serialization.load_pem_public_key(await get_msg(code))

# Trust and generate a key based on Alice's parameters
# Maybe should just follow the RFC 3526 parameters?
g = alice_trusted.public_numbers().parameter_numbers.g
p = alice_trusted.public_numbers().parameter_numbers.p

parameters = dh.DHParameterNumbers(p, g).parameters()
bob = parameters.generate_private_key()

pem = bob.public_key().public_bytes(
  encoding=serialization.Encoding.PEM,
  format=serialization.PublicFormat.SubjectPublicKeyInfo
)

await send_msg(pem,code)

s = bob.exchange(alice_trusted)
print(f"Compute the shared secret as s=(A^b) mod p={s.hex()}")

secret_message = await get_msg(code)

print(xor(base64.b64decode(secret_message),s).decode('utf-8'))